In [0]:
from pyspark.sql.functions import coalesce, col, when

eh = spark.table("personal_finance.silver.consolidated_equity_holdings")
yi = spark.table("personal_finance.bronze.yfinance_company_info")

df = (
    eh.alias("eh")
    .join(yi.alias("yi"), col("eh.ISIN") == col("yi.ISIN"))
    .select(
        "eh.*",
        coalesce(col("yi.currentPrice"), col("yi.previousClose")).alias("LatestTradedPrice"),
        col("yi.previousClose").alias("PreviousClose"),
        (col("eh.TotalQuantity") * coalesce(col("yi.currentPrice"), col("yi.previousClose"))).alias("HoldingValue"),
        (coalesce(col("yi.currentPrice"), col("yi.previousClose")) - col("eh.WeightedAverageBuyPrice")).alias("Returns"),
        (
            (coalesce(col("yi.currentPrice"), col("yi.previousClose")) - col("eh.WeightedAverageBuyPrice")) /
            when(col("eh.WeightedAverageBuyPrice") != 0, col("eh.WeightedAverageBuyPrice"))
        ).alias("ReturnsPercentage"),
        col("yi.trailingPE").cast("double").alias("peRatio"),
        col("yi.priceToBook").cast("double").alias("PbRatio")
    )
)

In [0]:
df.write.mode("overwrite").format("delta").saveAsTable("personal_finance.gold.equity_holdings")
display(df)